## 2016 Election Project 

This notebook is intended to document my data processing throughout this project. I'll be poking around and modifying my data in this file. The data I am starting out with are transcripts of the presidential debates from the 2016 US Election between Hillary Clinton and Donald Trump. The transcripts were taken from UCSB's American Presidency Project, and the citation for each of the transcripts will be included both above, and as a part of the data I process.

To begin, I have three transcripts from the three presidential debates.

**I plan to use other speeches as well, but I think I will use their individual speeches AFTER they have both been chosen as their party's candidates.**

In [52]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import pandas as pd
import glob
import os


In [57]:
os.chdir('/Users/Paige/Documents/Data_Science/2016-Election-Project/data/Debates')
files = glob.glob("*.txt")
files

['10-19-16.txt', '10-9-16.txt', '9-26-16.txt']

In [58]:
transcripts = []
for f in files:
    fi = open(f, 'r')
    txt = fi.read()
    fi.close
    transcripts.append(txt)

In [60]:
print(transcripts[0][:200])

PARTICIPANTS:
Former Secretary of State Hillary Clinton (D) and
Businessman Donald Trump (R)
MODERATOR:
Chris Wallace (Fox News)

WALLACE: Good evening from the Thomas and Mack Center at the Universit


In [62]:
print(transcripts[1][:200])

PARTICIPANTS:
Former Secretary of State Hillary Clinton (D) and
Businessman Donald Trump (R)
MODERATORS:
Anderson Cooper (CNN) and
Martha Raddatz (ABC News)

RADDATZ: Ladies and gentlemen the Republic


In [63]:
print(transcripts[2][:200])

PARTICIPANTS:
Former Secretary of State Hillary Clinton (D) and
Businessman Donald Trump (R)
MODERATOR:
Lester Holt (NBC News)

HOLT: Good evening from Hofstra University in Hempstead, New York. I'm L


In [69]:
t0sents = nltk.sent_tokenize(transcripts[0])
t1sents = nltk.sent_tokenize(transcripts[1])
t2sents = nltk.sent_tokenize(transcripts[2])

In [66]:
t0sents[:3]

['PARTICIPANTS:\nFormer Secretary of State Hillary Clinton (D) and\nBusinessman Donald Trump (R)\nMODERATOR:\nChris Wallace (Fox News)\n\nWALLACE: Good evening from the Thomas and Mack Center at the University of Nevada, Las Vegas.',
 "I'm Chris Wallace of Fox News, and I welcome you to the third and final of the 2016 presidential debates between Secretary of State Hillary Clinton and Donald J. Trump.",
 'This debate is sponsored by the Commission on Presidential Debates.']